In [1]:
# Importing required libraries

# !pip install numpy
# !pip install pandas
# !pip install openpyxl
# !pip install plotly-express
# !pip install nbformat==4.2.0
# !pip install ipykernel
# !pip install --upgrade nbformat
# !pip install matplotlib
# !pip install chart-studio

!import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
import numpy as np   # For numericall array 
import pandas as pd  # For dataframe calculations

'import' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# Reading the input data
df = pd.read_excel("VaR.xlsx", 
                    sheet_name = "Q 4", engine = "openpyxl")

# Selecting only required columns and renaming columns for just a convinience
df = df[["date", "Portfolio", "market rate", "market rate.1"]]

df = df.rename(columns = {"date":"Date", "market rate":"Currency 1",
                                        "market rate.1":"Currency 2"})
print("Sampling the head of the dataset")
display(df.head())
print("Sampling the tail of dataset")
display(df.tail())

Sampling the head of the dataset


,Date,Portfolio,Currency 1,Currency 2
0,2019-11-14,FXTEST,1.168443,0.886564
1,2019-11-13,FXTEST,1.165977,0.884330
2,2019-11-12,FXTEST,1.166031,0.883470
3,2019-11-11,FXTEST,1.166902,0.877539
4,2019-11-08,FXTEST,1.160726,0.877659


Sampling the tail of dataset


,Date,Portfolio,Currency 1,Currency 2
255,2018-11-20,FXTEST,1.123899,0.912825
256,2018-11-19,FXTEST,1.247645,0.910705
257,2018-11-16,FXTEST,1.126722,0.900414
258,2018-11-15,FXTEST,1.129599,0.905879
259,2018-11-14,FXTEST,1.149980,0.895255


# Normality Test (Optional for this project)

### Shapiro check

In [3]:
from scipy.stats import shapiro
stat, p = shapiro(df["Currency 1"])

# interpret
alpha = 0.05
if p > alpha:
    msg = 'Sample looks Gaussian (fail to reject H0)'
else:
    msg = 'Sample does not look Gaussian (reject H0)'

result_mat = [
    ['Length of the sample data', 'Test Statistic', 'p-value', 'Comments'],
    [len(df["Currency 1"]), stat, p, msg]
]

pd.DataFrame(result_mat)

,0,1,2,3
0,Length of the sample data,Test Statistic,p-value,Comments
1,260,0.535035,0.0,Sample does not look Gaussian (reject H0)


In [4]:
from scipy.stats import shapiro
stat, p = shapiro(df["Currency 2"])

# interpret
alpha = 0.05
if p > alpha:
    msg = 'Sample looks Gaussian (fail to reject H0)'
else:
    msg = 'Sample does not look Gaussian (reject H0)'

result_mat = [
    ['Length of the sample data', 'Test Statistic', 'p-value', 'Comments'],
    [len(df["Currency 1"]), stat, p, msg]
]

pd.DataFrame(result_mat)

,0,1,2,3
0,Length of the sample data,Test Statistic,p-value,Comments
1,260,0.977346,0.00037,Sample does not look Gaussian (reject H0)


### Normal test

In [5]:
from scipy.stats import normaltest

stat, p = normaltest(df["Currency 1"])

# interpret
alpha = 0.05
if p > alpha:
    msg = 'Sample looks Gaussian (fail to reject H0)'
else:
    msg = 'Sample does not look Gaussian (reject H0)'

result_mat = [
    ['Length of the sample data', 'Test Statistic', 'p-value', 'Comments'],
    [len(df["Currency 2"]), stat, p, msg]
]

pd.DataFrame(result_mat)

,0,1,2,3
0,Length of the sample data,Test Statistic,p-value,Comments
1,260,357.78146,0.0,Sample does not look Gaussian (reject H0)


In [6]:
from scipy.stats import normaltest

stat, p = normaltest(df["Currency 2"])

# interpret
alpha = 0.05
if p > alpha:
    msg = 'Sample looks Gaussian (fail to reject H0)'
else:
    msg = 'Sample does not look Gaussian (reject H0)'

result_mat = [
    ['Length of the sample data', 'Test Statistic', 'p-value', 'Comments'],
    [len(df["Currency 2"]), stat, p, msg]
]

pd.DataFrame(result_mat)

,0,1,2,3
0,Length of the sample data,Test Statistic,p-value,Comments
1,260,15.008905,0.000551,Sample does not look Gaussian (reject H0)


# VaR Implementation

In [7]:
class VaR:
    """" 
    The code is designed to claclate historical VaR (1 day with .99 confidence level) for the FX portfolio assuming no corrolation between currencies.
    """
    
    def __init__(self, df, list_curre, list_values, shift_type):

        self.df               = df                                    # Date, portfolio and market rate are loaded as dataframe               
        self.list             = []                                    # Empty list to collect the values of shifts
        self.shift_type       = shift_type                            # Shift types e.g., Absloute, Relative, Logarithmic
        self.list_curre       = list_curre                            # List of currencies e.g., Currency 1	, Currency 2	
        self.list_values      = list_values                           # SPOT Portfolio values for each currency
        self.Parial_p_and_l   = pd.DataFrame([0]*self.df.shape[0], 
                                                columns = ["p & l"])  # Zero vector to store the values of partial PnL

    # The below method is calculating the VaR while looping over each currency 
    def Value_at_risk(self):

        for Asset, Value in zip(self.list_curre, self.list_values):

            # For each given currency, the below part of code calculates the 1-day shift
            # for each given type of shift e.g.,  Absloute, Relative, Logarithmic

            for i in range(self.df.shape[0] - 1):
                    
                    # calculating 1-day shift while the shift type is Absloute
                    if  self.shift_type == "Absloute":

                        self.list.append(df[Asset].iloc[i] - df[Asset].iloc[i+1])

                    # calculating 1-day shift while the shift type is Logarithmic
                    elif self.shift_type== "Logarithmic":

                        self.list.append(np.exp(np.log(df[Asset].iloc[i]/df[Asset].iloc[i+1])) - 1)

                    # calculating 1-day shift while the shift type is Relative
                    elif self.shift_type== "Relative":

                        self.list.append((df[Asset].iloc[i]- df[Asset].iloc[i+1])/(df[Asset].iloc[i+1]))

            # Just for the convinience, the last element will be zero
            self.list.append(0)
            
            # Partial PnL is collecting share of each currency
            self.Parial_p_and_l["p & l"]   = self.Parial_p_and_l["p & l"] + pd.Series(self.list, name = "P & L")*Value

            # Clearing the list to be used again
            self.list = []
            
        # Sorting the obtained total PnL to select the second and third worst PnL    
        self.Parial_p_and_l = self.Parial_p_and_l.sort_values(by = "p & l", ascending = False)
        
        # According to the methodology. 
        self.VaR            = 0.4*self.Parial_p_and_l["p & l"].iloc[-2] + 0.6*self.Parial_p_and_l["p & l"].iloc[-3] 

        return self.VaR
    

    # The below method is calculating the VaR while looping over each currency 
    def p_and_l(self):

        self.loss_distribution = pd.DataFrame([], columns = ["P & L"])

        for Asset, Value in zip(self.list_curre, self.list_values):

            # For each given currency, the below part of code calculates the 1-day shift
            # for each given type of shift e.g.,  Absloute, Relative, Logarithmic

            for i in range(self.df.shape[0] - 1):
                    
                    # calculating 1-day shift while the shift type is Absloute
                    if  self.shift_type == "Absloute":

                        self.list.append(df[Asset].iloc[i] - df[Asset].iloc[i+1])

                    # calculating 1-day shift while the shift type is Logarithmic
                    elif self.shift_type== "Logarithmic":

                        self.list.append(np.exp(np.log(df[Asset].iloc[i]/df[Asset].iloc[i+1])) - 1)

                    # calculating 1-day shift while the shift type is Relative
                    elif self.shift_type== "Relative":

                        self.list.append((df[Asset].iloc[i]- df[Asset].iloc[i+1])/(df[Asset].iloc[i+1]))

            # Just for the convinience, the last element will be zero
            self.list.append(0)
            
            # Partial PnL is collecting share of each currency
            self.loss_distribution   = pd.concat([self.loss_distribution, pd.Series(self.list, name = str(Asset) + "P & L")], axis = 1)

            # Clearing the list to be used again
            self.list = []

        return self.loss_distribution

In [8]:
# Example usage
list_curre  = ["Currency 1", "Currency 2"]
list_values = [153084.81   ,  95891.51   ] 

Value_at_Risk_abs = VaR(df, list_curre, list_values, "Absloute")
Value_at_Risk_rel = VaR(df, list_curre, list_values, "Relative")
Value_at_Risk_log = VaR(df, list_curre, list_values, "Logarithmic")


Loss_distributions = Value_at_Risk_abs.p_and_l()

In [9]:
# x1 = Loss_distributions["Currency 1P & L"]
# x2 = Loss_distributions["Currency 2P & L"]

# hist_data = [x1, x2,]

# group_labels = ['Currency 1P & L', 'Currency 2P & L',]
# colors       = ['#A56CC1', '#A6ACEC', ]

# # Create distplot with curve_type set to 'normal'
# fig = ff.create_distplot(hist_data, group_labels, colors=colors,
#                         bin_size=.001, show_rug=False, show_curve=False)

# # Add title
# fig.update_layout(title_text='Hist and Curve Plot')
# # fig.update_layout(xaxis_range=[-0.05,0.05])
# fig.show()

In [10]:
print("The calculated Value at Risk (Var 1-d) while the shift type is Absloute is    :{:.2f}".format(Value_at_Risk_abs.Value_at_risk()))
print("The calculated Value at Risk (Var 1-d) while the shift type is Relative is    :{:.2f}".format(Value_at_Risk_rel.Value_at_risk()))
print("The calculated Value at Risk (Var 1-d) while the shift type is Logarithmic is :{:.2f}".format(Value_at_Risk_log.Value_at_risk()))

The calculated Value at Risk (Var 1-d) while the shift type is Absloute is    :-15818.67
The calculated Value at Risk (Var 1-d) while the shift type is Relative is    :-13572.73
The calculated Value at Risk (Var 1-d) while the shift type is Logarithmic is :-13572.73
